In [1]:
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Connect to Database

In [3]:
#Connect to PgAdmin
engine = psycopg2.connect(
    database="project-data",
    user = "postgres",
    password = "Post23!Fnord",
    host = "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    port = '5432'
    )

cursor = engine.cursor()


In [4]:
#Pull combined data table
query = "SELECT * from combined_data"

df = pd.read_sql(query, con=engine)

In [5]:
df.head()

,country_name,region,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,...,ladder_2010,ladder_2012,ladder_2013,ladder_2014,ladder_2015,ladder_2016,ladder_2017,ladder_2018,ladder_2019,ladder_2020
0,Brazil,Latin America and Caribbean,6.330,0.043,6.415,6.245,9.577,0.882,66.601,0.804,...,6.837,7.038,7.140,6.981,6.547,6.375,6.333,6.191,6.451,6.110
1,India,South Asia,3.819,0.026,3.869,3.769,8.755,0.603,60.633,0.893,...,4.989,4.635,4.428,4.424,4.342,4.179,4.046,3.818,3.249,4.225
2,Russia,Commonwealth of Independent States,5.477,0.033,5.541,5.413,10.189,0.903,64.703,0.718,...,5.385,5.389,5.537,6.037,5.996,5.855,5.579,5.514,5.441,5.495
3,South Africa,Sub-Saharan Africa,4.956,0.060,5.074,4.839,9.403,0.860,56.904,0.749,...,4.652,4.931,3.661,4.828,4.887,4.770,4.514,4.884,5.035,4.947
4,Mexico,Latin America and Caribbean,6.317,0.053,6.420,6.213,9.859,0.831,68.597,0.862,...,6.802,6.910,7.443,6.680,6.236,6.824,6.410,6.550,6.432,5.964


# Clean master table for Machine Learning

In [6]:
#column names
list(df)

['country_name',
 'region',
 'ladder_score',
 'stand_err',
 'up_whisker',
 'low_whisker',
 'log_gdp_per_cap',
 'social_support',
 'healty_life_exp',
 'freedom',
 'generosity',
 'percept_corrupt',
 'ladder_score_distopia',
 'expby_log_gdp_per_cap',
 'expby_social_support',
 'expby_healty_life_exp',
 'expby_freedom',
 'expby_generosity',
 'expby_percept_corrupt',
 'distopia_plus_resid',
 'ac_both_sexes',
 'ac_male',
 'ac_female',
 'population_2021',
 'covid_total_cases',
 'covid_new_cases',
 'covid_total_deaths',
 'covid_new_deaths',
 'covid_total_recovered',
 'covid_new_recovered',
 'covid_active_cases',
 'covid_serious_critical',
 'covid_cases_per_mil',
 'covid_deaths_per_mil',
 'covid_total_tests',
 'covid_tests_per_mil',
 'covid_who_region',
 'population_2020',
 'land_area_skm',
 'density_skm',
 'meat_consumption',
 'median_age',
 'screen_time_avg',
 'suicide_rate',
 'ladder_2010',
 'ladder_2012',
 'ladder_2013',
 'ladder_2014',
 'ladder_2015',
 'ladder_2016',
 'ladder_2017',
 'ladde

In [7]:
#Drop columns not needed for analysis
df = df.drop(columns = "population_2020")
df = df.drop(columns = "ladder_2010")
df = df.drop(columns = "ladder_2012")
df = df.drop(columns = "ladder_2013")
df = df.drop(columns = "ladder_2014")
df = df.drop(columns = "ladder_2015")
df = df.drop(columns = "ladder_2016")
df = df.drop(columns = "ladder_2017")
df = df.drop(columns = "ladder_2018")
df = df.drop(columns = "ladder_2019")
df = df.drop(columns = "ladder_2020")
df = df.drop(columns = "country_name")
df = df.drop(columns = "up_whisker")
df = df.drop(columns = "low_whisker")
df = df.drop(columns = "expby_log_gdp_per_cap")
df = df.drop(columns = "expby_social_support")
df = df.drop(columns = "expby_healty_life_exp")
df = df.drop(columns = "expby_freedom")
df = df.drop(columns = "expby_generosity")
df = df.drop(columns = "expby_percept_corrupt")
df = df.drop(columns = "stand_err")
df = df.drop(columns = "distopia_plus_resid")
df = df.drop(columns = "ac_both_sexes")
df = df.drop(columns = "region")

In [8]:
# view total null values for columns
df.isna().sum()

ladder_score                0
log_gdp_per_cap             0
social_support              0
healty_life_exp             0
freedom                     0
generosity                  0
percept_corrupt             0
ladder_score_distopia       0
ac_male                     8
ac_female                   8
population_2021            14
covid_total_cases          14
covid_new_cases           146
covid_total_deaths         17
covid_new_deaths          147
covid_total_recovered      17
covid_new_recovered       147
covid_active_cases         17
covid_serious_critical     56
covid_cases_per_mil        14
covid_deaths_per_mil       17
covid_total_tests          23
covid_tests_per_mil        23
covid_who_region           14
land_area_skm              11
density_skm                11
meat_consumption           12
median_age                 10
screen_time_avg           108
suicide_rate               12
dtype: int64

In [9]:
#compare NA values to dataset row count
len(df)

149

In [10]:
#drop columns with high NA count (Na count > 75)
df = df.drop(columns="covid_new_cases")
df = df.drop(columns="covid_new_deaths")
df = df.drop(columns="covid_new_recovered")
df = df.drop(columns="screen_time_avg")

In [11]:
# review total null values for columns
df.isna().sum()

ladder_score               0
log_gdp_per_cap            0
social_support             0
healty_life_exp            0
freedom                    0
generosity                 0
percept_corrupt            0
ladder_score_distopia      0
ac_male                    8
ac_female                  8
population_2021           14
covid_total_cases         14
covid_total_deaths        17
covid_total_recovered     17
covid_active_cases        17
covid_serious_critical    56
covid_cases_per_mil       14
covid_deaths_per_mil      17
covid_total_tests         23
covid_tests_per_mil       23
covid_who_region          14
land_area_skm             11
density_skm               11
meat_consumption          12
median_age                10
suicide_rate              12
dtype: int64

In [12]:
#drop all rows containing NA values
df = df.dropna()

In [13]:
# view total null values for columns
df.isna().sum()

ladder_score              0
log_gdp_per_cap           0
social_support            0
healty_life_exp           0
freedom                   0
generosity                0
percept_corrupt           0
ladder_score_distopia     0
ac_male                   0
ac_female                 0
population_2021           0
covid_total_cases         0
covid_total_deaths        0
covid_total_recovered     0
covid_active_cases        0
covid_serious_critical    0
covid_cases_per_mil       0
covid_deaths_per_mil      0
covid_total_tests         0
covid_tests_per_mil       0
covid_who_region          0
land_area_skm             0
density_skm               0
meat_consumption          0
median_age                0
suicide_rate              0
dtype: int64

In [14]:
#length of final dataset for analysis 
len(df)

84

# Random Forest: Find the 10 variables that impact happiness scores the most 

In [15]:
# Create columns and target variables
columns = [
    "country_name", "region", "ladder_score", "stand_err",
    "up_whisker", "low_whisker", "log_gdp_per_cap", "social_support",
    "expby_social_support", "freedom", "generosity", "percept_corrupt",
    "ladder_score_distopia", "expby_log_gdp_per_cap", "expby_social_support", "expby_healty_life_exp",
    "expby_freedom", "expby_generosity", "expby_percept_corrupt", "distopia_plus_resid", "ac_both_sexes", "ac_male",
    "ac_female", "population_2021", "covid_total_cases", "covid_new_cases", "covid_total_deaths", "covid_total_recovered",
    "covid_new_recovered", "covid_active_cases", "covid_serious_critical", "covid_cases_per_mil", "covid_deaths_per_mil",
    "covid_total_tests", "covid_tests_per_mil", "covid_who_region", "population_2020", "land_area_skm", "land_area_skm", 
    "density_skm", "meat_consumption", "median_age", "screen_time_avg", "suicide_rate", "ladder_2010", "ladder_2012", 
    "ladder_2013", "ladder_2014", "ladder_2015", "ladder_2016", "ladder_2017", "ladder_2018", "ladder_2019", "ladder_2020"
]

target = ["ladder_score"]

In [16]:
#Convert variable type for target to non-continuous dtype
df = df.astype({'ladder_score':'int'})

df.head()

,ladder_score,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,ac_male,ac_female,...,covid_cases_per_mil,covid_deaths_per_mil,covid_total_tests,covid_tests_per_mil,covid_who_region,land_area_skm,density_skm,meat_consumption,median_age,suicide_rate
0,6,9.577,0.882,66.601,0.804,-0.071,0.756,2.43,11.65,3.22,...,13716.0,464.0,13206188.0,62085.0,Americas,8515770.0,25.06,82.4,32.6,82.4
1,3,8.755,0.603,60.633,0.893,0.089,0.774,2.43,9.06,1.89,...,1466.0,30.0,22149351.0,16035.0,South-EastAsia,3287263.0,454.94,5.2,28.1,5.2
2,5,10.189,0.903,64.703,0.718,-0.111,0.845,2.43,17.99,4.31,...,5974.0,100.0,29716907.0,203623.0,Europe,17098242.0,8.82,51.0,39.6,51.0
3,4,9.403,0.860,56.904,0.749,-0.067,0.860,2.43,15.74,3.46,...,9063.0,162.0,3149807.0,53044.0,Africa,1219090.0,47.63,39.0,27.1,39.0
4,6,9.859,0.831,68.597,0.862,-0.147,0.799,2.43,8.13,2.18,...,3585.0,391.0,1056915.0,8189.0,Americas,1964375.0,64.91,58.6,28.3,58.6


In [17]:
# Create our features
X = pd.get_dummies(df.drop(columns="ladder_score"))


# Create our target
y = df["ladder_score"]

In [18]:
X.describe()

,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,ac_male,ac_female,population_2021,...,density_skm,meat_consumption,median_age,suicide_rate,covid_who_region_Africa,covid_who_region_Americas,covid_who_region_EasternMediterranean,covid_who_region_Europe,covid_who_region_South-EastAsia,covid_who_region_WesternPacific
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,8.400000e+01,84.000000,84.000000,8.400000e+01,...,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,9.627226,0.825512,65.973690,0.795107,-0.049690,0.753190,2.430000e+00,9.467762,2.652560,5.162632e+07,...,160.061786,48.544048,31.216667,48.544048,0.190476,0.214286,0.142857,0.357143,0.047619,0.047619
std,1.003613,0.112456,5.847626,0.113524,0.130379,0.164949,4.467564e-15,6.536607,2.040093,1.544042e+08,...,293.808761,33.012781,8.604648,33.012781,0.395035,0.412790,0.352029,0.482035,0.214238,0.214238
min,6.958000,0.463000,50.102000,0.382000,-0.288000,0.179000,2.430000e+00,0.000000,0.000000,5.414480e+05,...,2.970000,5.100000,16.500000,5.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.053750,0.773750,61.999500,0.740750,-0.147500,0.720000,2.430000e+00,3.605000,0.777500,6.523238e+06,...,45.045000,21.225000,24.200000,21.225000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.659500,0.850500,67.000000,0.818000,-0.075000,0.802000,2.430000e+00,8.950000,2.295000,1.722635e+07,...,87.110000,42.500000,29.950000,42.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10.372250,0.903750,69.776500,0.877500,0.037250,0.857000,2.430000e+00,15.580000,4.420000,4.115598e+07,...,144.892500,70.400000,39.375000,70.400000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,11.647000,0.954000,75.100000,0.970000,0.311000,0.939000,2.430000e+00,20.610000,6.340000,1.381345e+09,...,2012.100000,145.900000,47.300000,145.900000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
# Check the balance of our target values
y.value_counts()

5    26
4    22
6    21
7    11
3     3
2     1
Name: ladder_score, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({5: 17, 4: 16, 6: 18, 7: 9, 2: 1, 3: 2})

In [21]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [22]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.48888888888888893

In [23]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[0, 1, 0, 0, 0],
       [0, 4, 2, 0, 0],
       [0, 2, 4, 3, 0],
       [0, 0, 2, 1, 0],
       [0, 0, 0, 0, 2]], dtype=int64)

In [24]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          3       0.00      0.00      1.00      0.00      0.00      0.00         1
          4       0.57      0.67      0.80      0.62      0.73      0.53         6
          5       0.50      0.44      0.67      0.47      0.54      0.29         9
          6       0.25      0.33      0.83      0.29      0.53      0.26         3
          7       1.00      1.00      1.00      1.00      1.00      1.00         2

avg / total       0.51      0.52      0.78      0.51      0.61      0.41        21



In [25]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.09248926923624289, 'freedom'),
 (0.08195228025005041, 'healty_life_exp'),
 (0.0710836330418065, 'log_gdp_per_cap'),
 (0.06937734100667901, 'meat_consumption'),
 (0.06934817566674269, 'percept_corrupt'),
 (0.05923589306111168, 'suicide_rate'),
 (0.05452746287980535, 'social_support'),
 (0.05426363854324505, 'covid_tests_per_mil'),
 (0.04370004253757945, 'ac_female'),
 (0.03835045962660014, 'generosity'),
 (0.03655847269240899, 'covid_deaths_per_mil'),
 (0.034342083870390845, 'covid_total_tests'),
 (0.027630970626381885, 'ac_male'),
 (0.02715553871853526, 'land_area_skm'),
 (0.02612964234942453, 'covid_active_cases'),
 (0.02559933603566062, 'covid_total_cases'),
 (0.024615275305677125, 'density_skm'),
 (0.024596223095569626, 'median_age'),
 (0.023335667907576775, 'covid_total_recovered'),
 (0.023246653903351202, 'covid_cases_per_mil'),
 (0.020762144467964466, 'covid_total_deaths'),
 (0.019995391312859743, 'population_2021'),
 (0.019880599387015856, 'covid_serious_critical'),
 (0.0092